# Getting the most out of NSIDC DAAC data: Discovering, Accessing, and Harmonizing Arctic remote sensing data

Here are the steps you will learn in this tutorial, which will be broken out into several Python Jupyter Notebooks:

Introduction
  
   * Learn the basics of Jupyter Lab and Jupyter Notebook
   * Introduce case study
   * Explore NSIDC data resources
   * Visualize time and area of interest using OpenAltimetry 
   
Customize and Access NSIDC DAAC Data   
   
   * Search for data programmatically by time and area of interest.
   * Determine subsetting and reformatting capabilities for our data of interest.
   * Access and customize data using NSIDC's API service.

Analyze 

   * Filter ICESat-2 data by quality flag 
   * Extracting gridded data along ICESat-2 tracks using xarray
   * Plot data with MODIS true color WMS endpoint. 
   * Cartopy is quite good for this.  Basemap is an alternative. OpenAltimetry plotting (3d plotly)



### Basics of Jupyter Lab and Jupyter Notebook

Before we delve into our Arctic sea ice data exploration, let's go over the basics of how to interact with these Jupyter Notebooks and the Jupyter Lab interface, as well as some background on our Jupyter Hub environment.

In [ ]:
# Hub
# Notebook vs Lab
# Execute a notebook
# Execute a cell
# Bulk comment, uncomment 

## Fast ice over the East Siberian Sea 

For this tutorial, we're interested in whether or not elevation data from the ICESat-2 mission can help us determine fast ice characteristics over the East Siberian Sea in conjunction with reflectance data.

There are a few data sets provided by NSIDC and LP DAAC that can help us with our analysis: 

ATL03
ATL07
MOD29
AMSR2 SI6

![](https://nsidc.org/sites/nsidc.org/files/images/atlas-beam-pattern.png)

See the ICESat-2 Data Sets page for a list of all ICESat-2 data set titles and IDs. 

From the ICESat-2 Data Sets page, you can find a link to each data set home page: </br>
ATL03: https://nsidc.org/data/atl03 </br> ATL07: https://nsidc.org/data/atl07

From that home page, several resources are available, including an online user guide (within the User Guide tab of the landing page): </br>
ATL03: https://nsidc.org/data/atl03?qt-data_set_tabs=3#qt-data_set_tabs </br> ATL07: https://nsidc.org/data/atl07?qt-data_set_tabs=3#qt-data_set_tabs

As well as a data dictionary with every data set variable described in detail: </br>
ATL03: https://nsidc.org/sites/nsidc.org/files/technical-references/ATL03-data-dictionary-v001.pdf </br> ATL07: https://nsidc.org/sites/nsidc.org/files/technical-references/ATL07-data-dictionary-v001.pdf

### Visualize time and area of interest

We're looking for a clear day across the coast, fast ice, and on to drift ice. 

Because we're mainly interested in ICESat-2 coverage, we'll use the OpenAltimetry application to easily visualize the coverage over our area of interest, as well as overlay MODIS reflectance imagery to observe cloud cover. 


OpenAltimetry

https://openaltimetry.org/data/icesat2/?annoId=119&shareKey=31a6dd4555dc1b8ccbd84fce02a63d37


